# Zadanie: fake news

In [192]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split as split

In [173]:
bodies = pd.read_csv("Data/bodies.csv")
stances = pd.read_csv("Data/stances.csv")

In [174]:
bodies = bodies.sort_values('Body ID')
bodies = bodies.reset_index()
bodies = bodies.drop(columns=['index'])

In [175]:
stances = stances.sort_values('Body ID')
stances = stances.reset_index()
stances = stances.drop(columns=['index'])

In [176]:
bodies

,Body ID,articleBody
0,0,A small meteorite crashed into a wooded area i...
1,4,Last week we hinted at what was to come as Ebo...
2,5,(NEWSER) – Wonder how long a Quarter Pounder w...
3,6,"Posting photos of a gun-toting child online, I..."
4,7,At least 25 suspected Boko Haram insurgents we...
...,...,...
1678,2528,Intelligence agencies hunting for identity of ...
1679,2529,"While Daleks ""know no fear"" and ""must not fear..."
1680,2530,More than 200 schoolgirls were kidnapped in Ap...
1681,2531,A Guantanamo Bay prisoner released last year a...


In [177]:
stances

,Headline,Body ID,Stance
0,Soldier shot near Canadian parliament building,0,unrelated
1,Caught a catfish record in Po: 127 kg and 2.67...,0,unrelated
2,Enormous 20-stone catfish caught with fishing ...,0,unrelated
3,Soldier shot at war memorial in Canada,0,unrelated
4,A soldier has been shot at Canada’s war memori...,0,unrelated
...,...,...,...
49967,Ann Arbor pizza delivery driver surprised with...,2532,agree
49968,"Luckiest Pizza Delivery Guy Ever Gets $2,000 Tip",2532,agree
49969,"Pizza delivery man gets $2,000 tip",2532,agree
49970,"Pizza delivery driver surprised with $2,000 tip",2532,agree


Do każdego tytułu dokleiłem treść artykułu:

In [178]:
j = 0
bodies_copy = bodies.copy()
for row_id in range(len(stances)):
    if stances.at[row_id, 'Body ID'] == bodies_copy.at[j, 'Body ID']:
        stances.at[row_id, 'Headline'] += ' ' + bodies_copy.at[j, 'articleBody']
    else:
        j += 1
        stances.at[row_id, 'Headline'] += ' ' + bodies_copy.at[j, 'articleBody']
        bodies_copy = bodies_copy.drop(index=(j-1))

Wektory słów:

In [185]:
stances_vectorizer = CountVectorizer()
stances_tfidf_transformer = TfidfTransformer()

In [189]:
st_vector = stances_vectorizer.fit_transform(stances.Headline)
st_transform = stances_tfidf_transformer.fit_transform(st_vector)

In [193]:
X_train, X_test, y_train, y_test = split(st_transform, stances.Stance, test_size = 0.2)

In [211]:
Xtext_train, Xtext_test, ytext_train, ytext_test = split(stances.Headline, stances.Stance, test_size = 0.2)

Naiwny klasyfikator Bayesowski:

In [204]:
clf = MultinomialNB().fit(X_train, y_train)

In [205]:
predicted = clf.predict(X_test)

In [206]:
np.mean(predicted == y_test)

0.751575787893947

In [234]:
from sklearn.pipeline import Pipeline
text_clf = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('clf', MultinomialNB()),
                     ])
text_clf = text_clf.fit(Xtext_train, ytext_train)

In [235]:
predicted = text_clf.predict(Xtext_test)
np.mean(predicted == ytext_test)

0.7529764882441221

In [237]:
from sklearn.model_selection import GridSearchCV
parameters = {'vect__ngram_range': [(1, 1), (1, 2)],
              'tfidf__use_idf': (True, False),
              'clf__alpha': (1e-2, 1e-3),
             }
gs_clf = GridSearchCV(text_clf, parameters, n_jobs=-1)
gs_clf = gs_clf.fit(Xtext_train, ytext_train)
predicted = gs_clf.predict(Xtext_test)

In [228]:
np.mean(predicted == ytext_test)

0.7563781890945472

Sieć neuronowa:

In [265]:
from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(solver='adam', alpha=1e-5, hidden_layer_sizes=(10, 2), random_state=1, max_iter = 300)
clf.fit(X_train, y_train)

MLPClassifier(alpha=1e-05, hidden_layer_sizes=(10, 2), max_iter=300,
              random_state=1)

In [266]:
predicted = clf.predict(X_test)
np.mean(predicted == y_test)

0.9196598299149574